## How to numba
Здесь я попытаюсь объяснить, как достичь максимальной производительности при использовании numba, как помочь компиллятору помочь нам

Так как большинство оптимизаций numba делает сама(перевод кода в машинный, векторизация…), то компиллятор должен понимать по вашему python коду, где и как можно его ускорить. Для того, чтобы ему это было проще сделать есть несколько основных реккомендаций:
1. Чем код проще --- тем он быстрее. Линейный код проще понимать
2. Старайтесь использовать меньше встроенных типов, numpy быстрее
3. Статическая типизация много быстрее

## Линейность кода
### SIMD
SIMD(Single Instruction, Multiple Data) - связка программных инструкций и аппаратных возможностей, которые позволяют выполнять одинаковые операции над несколькими входными данными одновременно.
Для примера: у многиъ процессоров есть большие регистры для арифметики чисел с плавающей точкой. Если мы используем `int`, то в один такой регистр можно поместить несколько чисел и одновременно выполнить операцию сложения/сравнения.(загрузка данных немного замедляется, то зато мы выигрываем в количестве произведенных действий, поэтому код получается быстрее)


In [11]:
import numpy as np
from numba import jit

@jit(nopython=True)
def sqdiff(x, y):
    out = np.empty_like(x)
    for i in range(x.shape[0]):
        out[i] = (x[i] - y[i])**2
    return out


x32 = np.linspace(1, 2, 10000, dtype=np.float32)
y32 = np.linspace(2, 3, 10000, dtype=np.float32)
sqdiff(x32, y32)


x64 = x32.astype(np.float64)
y64 = y32.astype(np.float64)
sqdiff(x64, y64)

array([1.        , 0.99999976, 1.        , ..., 1.        , 1.00000024,
       1.        ])

In [12]:
sqdiff.signatures

[(Array(float32, 1, 'C', False, aligned=True),
  Array(float32, 1, 'C', False, aligned=True)),
 (Array(float64, 1, 'C', False, aligned=True),
  Array(float64, 1, 'C', False, aligned=True))]

2 функции: одна для 32 битных, другая для 64 битных чисел. Можно заметить, что первая работает быстрее т.к. используется SIMD

In [ ]:
%timeit sqdiff(x32, y32)
%timeit sqdiff(x64, y64)

3.2 µs ± 545 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
4.79 µs ± 649 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


можно посмотреть на использование оптимизаций:

In [22]:
def find_instr(func, keyword, sig=0):
    asm = func.inspect_asm(func.signatures[sig])
    for line in asm.split('\n'):
        if keyword in line:
            print(line)

In [ ]:
find_instr(sqdiff, keyword='subp', sig=1)

	vsubpd	(%rax,%rdi,8), %ymm0, %ymm0
	vsubpd	32(%rax,%rdi,8), %ymm1, %ymm1
	vsubpd	64(%rax,%rdi,8), %ymm2, %ymm2
	vsubpd	96(%rax,%rdi,8), %ymm3, %ymm3
	vsubpd	128(%rax,%rdi,8), %ymm0, %ymm0
	vsubpd	160(%rax,%rdi,8), %ymm1, %ymm1
	vsubpd	192(%rax,%rdi,8), %ymm2, %ymm2
	vsubpd	224(%rax,%rdi,8), %ymm3, %ymm3
	vsubpd	(%rax,%rdi,8), %ymm0, %ymm0
	vsubpd	32(%rax,%rdi,8), %ymm1, %ymm1
	vsubpd	64(%rax,%rdi,8), %ymm2, %ymm2
	vsubpd	96(%rax,%rdi,8), %ymm3, %ymm3



### Branchfree code
numba использует LLVM, которая может оптимизировать код с использованием SIMD, но для того, чтобы компиллятор мог понять, где он может применить данную оптимизацию, ему нужно помочь. Так, например, если в цикле есть конструкции if…else…, то numba не сможет оптимизировать его под SIMD т.к. поток данных будет не равномерным и мы не сможем нормально заполнять регистры.


In [41]:
@jit(nopython=True)
def diff_branch(x, y):
    out = np.empty_like(x)
    for i in range(x.shape[0]):
        if x[i] > y[i]:
            out[i] = (x[i] - y[i])
        else:
            out[i] = (x[i] + y[i])**2
    return out

In [42]:
diff_branch(x32, y32)
diff_branch(x64, y64)

array([ 9.        ,  9.00119952,  9.00240056, ..., 24.9959995 ,
       24.9980009 , 25.        ])

Видимо, numba смог что-то соптимизировать, но количество инструкций заметно уменьшилось

In [43]:
find_instr(diff_branch, keyword='subp', sig=1)

	vsubpd	%ymm4, %ymm0, %ymm0
	vsubpd	%ymm5, %ymm1, %ymm1
	vsubpd	%ymm6, %ymm2, %ymm2
	vsubpd	%ymm7, %ymm3, %ymm3



### Numpy error model
Отсылаясь к предыдущему пункту, истоит использовать модель numpy т.к. например, деление на 0 в python использует exception, а в numpy просто возвращает nan/inf, поэтому numba оптимизирует последний код с учётом SIMD инструкций.

In [52]:
1 / 0

ZeroDivisionError: division by zero

In [51]:
@jit(nopython=True)
def frac_diff_py(x, y):
    out = np.empty_like(x)
    for i in range(x.shape[0]):
        out[i] = 2 * (x[i] - y[i]) / (x[i] + y[i])
    return out

frac_diff_py(x32, y32)
frac_diff_py(x64, y64)

array([-0.66666667, -0.66662216, -0.66657777, ..., -0.40003201,
       -0.40001604, -0.4       ])

Инструкций нет

In [53]:
find_instr(frac_diff_py, keyword='subp', sig=1)

In [55]:
@jit(nopython=True, error_model='numpy')
def frac_diff_nu(x, y):
    out = np.empty_like(x)
    for i in range(x.shape[0]):
        out[i] = 2 * (x[i] - y[i]) / (x[i] + y[i])
    return out

frac_diff_nu(x32, y32)
frac_diff_nu(x64, y64)

array([-0.66666667, -0.66662216, -0.66657777, ..., -0.40003201,
       -0.40001604, -0.4       ])

numba оптимизировал код

In [57]:
find_instr(frac_diff_nu, keyword='subp', sig=1)

	vsubpd	%ymm1, %ymm0, %ymm2
	vsubpd	%ymm1, %ymm0, %ymm2
	vsubpd	%ymm1, %ymm0, %ymm2
	vsubpd	%ymm1, %ymm0, %ymm2
	vsubpd	%ymm1, %ymm0, %ymm2


In [58]:
frac_diff_nu.inspect_types(pretty=True)

/usr/local/lib/python3.12/dist-packages/numba/core/annotations/pretty_annotate.py:7: FutureWarning: The pretty_annotate functionality is experimental and might change API
  warn("The pretty_annotate functionality is experimental and might change API",


label 0
"x = arg(0, name=x) :: array(float32, 1d, C)"
"y = arg(1, name=y) :: array(float32, 1d, C)"
"2: def frac_diff_nu(x, y):"
$4load_global.0 = global(np: <module 'numpy' from '/usr/local/lib/python3.12/dist-packages/numpy/__init__.py'>) :: Module(<module 'numpy' from '/usr/local/lib/python3.12/dist-packages/numpy/__init__.py'>)
"$14load_attr.2 = getattr(value=$4load_global.0, attr=empty_like) :: Function(<built-in function empty_like>)"
del $4load_global.0
"out = call $14load_attr.2(x, func=$14load_attr.2, args=[Var(x, ipython-input-2667628562.py:1)], kws=(), vararg=None, varkwarg=None, target=None) :: (Array(float32, 1, 'C', False, aligned=True), omitted(default=None)) -> array(float32, 1d, C)"
del $14load_attr.2
$46load_global.5 = global(range: <class 'range'>) :: Function(<class 'range'>)
"$58load_attr.8 = getattr(value=x, attr=shape) :: UniTuple(int64 x 1)"


Так же можно помогать numba еще и в определении типов, заметим, что константа 2 -- int, это немного замедляет вычисления т.к. нужно каждый раз приводить к типу остальных данных(float)

In [60]:
def frac_diff_nu2(x, y):
    out = np.empty_like(x)
    for i in range(x.shape[0]):
        out[i] = np.float32(2) * (x[i] - y[i]) / (x[i] + y[i])
    return out

In [62]:
print("Python:")
%timeit frac_diff_py(x32, y32)
%timeit frac_diff_py(x64, y64)

print("Numpy error model")
%timeit frac_diff_nu(x32, y32)
%timeit frac_diff_nu(x64, y64)

print("Optimized numpy")
%timeit frac_diff_nu2(x32, y32)
%timeit frac_diff_nu2(x64, y64)

Python:
22.5 µs ± 613 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
18.1 µs ± 2.73 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)
Numpy error model
14.5 ms ± 2.52 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
16.2 ms ± 2.48 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
Optimized numpy
14.8 ms ± 2.35 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
16 ms ± 2.55 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Fastmath
Иногда, numba выгоднее преобразовывать операции для того, чтобы использовать векторные вычисления, менять порядок вычисления в суммах, немного теряя в точности, выигрывать в производительности.

In [68]:
@jit(nopython=True, error_model='numpy')
def compute_normal(x):
    out = 0.
    for i in range(x.shape[0]):
        out += np.sqrt(x[i] * x[i] + 1.0) / (x[i] + 1.0)
    return out

@jit(nopython=True, fastmath=True, error_model='numpy')
def compute_fast(x):
    out = 0.
    for i in range(x.shape[0]):
        out += np.sqrt(x[i] * x[i] + 1.0) / (x[i] + 1.0)
    return out

compute_normal(x64)
compute_fast(x64)

7226.773259159489

В два раза быстрее

In [69]:
%timeit compute_normal(x64)
%timeit compute_fast(x64)

38.3 µs ± 142 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
19.5 µs ± 272 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
